Leung Wai Liu <br>
JPMC <br>
July 25, 2022 <br>
Twitter Relabelling

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
combined_df = pd.read_csv('logs/combined_final.tsv', sep='\t')
combined_df

,Word,POS Label,NER Label,Tweet
0,poutine,PROPN,O,rt @user1787 : i asked siri to remind me about...
1,friday,PROPN,O,rt @user912 : you know it 's friday when the p...
2,@user444,PROPN,O,mikel obi & his girl olga diyachenko spend hol...
3,@user1598,PROPN,O,""" @user1598 : super junior ’s siwon transforms..."
4,gospel,PROPN,O,""" do n't move on from the gospel , move into t..."
...,...,...,...,...
1013,@user70,PROPN,O,rt @user2041 : @user70 happy birthday man ! le...
1014,@user660,PROPN,O,rt @user1157 : happy birthday @user660 ! url1209
1015,@user2046,PROPN,O,@user2046 further you do the swedish social se...
1016,@user2242,PROPN,O,@user2046 further you do the swedish social se...


In [4]:
# Changing all @user to be B-PER for now

users_change_df = combined_df

for index, row in users_change_df.iterrows():
    particular_word = row['Word']
    if '@user' in particular_word:
        row['NER Label'] = 'B-PER'


In [5]:
users_change_df

,Word,POS Label,NER Label,Tweet
0,poutine,PROPN,O,rt @user1787 : i asked siri to remind me about...
1,friday,PROPN,O,rt @user912 : you know it 's friday when the p...
2,@user444,PROPN,B-PER,mikel obi & his girl olga diyachenko spend hol...
3,@user1598,PROPN,B-PER,""" @user1598 : super junior ’s siwon transforms..."
4,gospel,PROPN,O,""" do n't move on from the gospel , move into t..."
...,...,...,...,...
1013,@user70,PROPN,B-PER,rt @user2041 : @user70 happy birthday man ! le...
1014,@user660,PROPN,B-PER,rt @user1157 : happy birthday @user660 ! url1209
1015,@user2046,PROPN,B-PER,@user2046 further you do the swedish social se...
1016,@user2242,PROPN,B-PER,@user2046 further you do the swedish social se...


In [6]:
users_change_df.to_csv('logs/users_changed.tsv', sep='\t', index=False)

In [7]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [16]:
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased', add_prefix_space=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [24]:
# WORDNINJA uses the virterbi probability to determine where to effectively split the word
import wordninja

In [60]:
import spacy
nlp = spacy.load('en_core_web_lg')
nlp2 = spacy.load('en_core_web_lg')


In [79]:
from collections import Counter

In [88]:
change_hashtags = users_change_df

In [89]:
for index, row in change_hashtags.iterrows():
    particular_word = row['Word']
    if '#' in particular_word:
        particular_word = particular_word[1:]
        encode = wordninja.split(particular_word)
        encode = ' '.join(encode)


        # encoding into nlp
        doc = nlp(encode)
        doc2 = nlp2(particular_word)
        ents = [(e.text, e.ent_iob_, e.ent_type_, e.pos_) for e in doc]
        ents2 = [(f.text, f.ent_iob_, f.ent_type_, f.pos_) for f in doc2]

        label = []
        pos = []
        if ents2[0][2] != '':
            label.append(ents2[0][2])
        
        for numents in range(len(ents)):
            if ents[numents][2] != '':
                label.append(ents[numents][2])
            pos.append(ents[numents][3])
        

        # Filtering
        if label != []:
            label_counter = Counter(label)
            label_most_common = (label_counter.most_common(1))[0][0]
            
            # checking the label itself and then labeling it
            if label_most_common == 'PERSON':
                row['NER Label'] = 'B-PER'
            elif label_most_common == 'FAC':
                row['NER Label'] = 'B-LOC'
            elif label_most_common == 'NORP':
                row['NER Label'] = 'B-ORG'
            elif label_most_common == 'ORG':
                row['NER Label'] = 'B-ORG'
            elif label_most_common != 'MONEY' or label_most_common != 'QUANTITY':
                row['NER Label'] = 'B-MISC'
    
        else: 
            # Looking through pos taggings
            if 'PROPN' in pos:
                row['NER Label'] = 'B-MISC'
            elif 'NOUN' in pos:
                row['NER Label'] = 'B-MISC'
            

        
        

In [90]:
change_hashtags.to_csv('logs/changed_hashtags.tsv', sep='\t', index=False)

In [91]:
change_hashtags

,Word,POS Label,NER Label,Tweet
0,poutine,PROPN,B-MISC,rt @user1787 : i asked siri to remind me about...
1,friday,PROPN,B-MISC,rt @user912 : you know it 's friday when the p...
2,@user444,PROPN,B-MISC,mikel obi & his girl olga diyachenko spend hol...
3,@user1598,PROPN,B-MISC,""" @user1598 : super junior ’s siwon transforms..."
4,gospel,PROPN,B-MISC,""" do n't move on from the gospel , move into t..."
...,...,...,...,...
1013,@user70,PROPN,B-PER,rt @user2041 : @user70 happy birthday man ! le...
1014,@user660,PROPN,B-PER,rt @user1157 : happy birthday @user660 ! url1209
1015,@user2046,PROPN,B-PER,@user2046 further you do the swedish social se...
1016,@user2242,PROPN,B-PER,@user2046 further you do the swedish social se...


In [92]:
change_everything = change_hashtags

In [93]:
for index, row in change_everything.iterrows():
    particular_word = row['Word']
    # Filtering everything else
    if '#' not in particular_word and '@user' not in particular_word:
        encode = wordninja.split(particular_word)
        encode = ' '.join(encode)


        # encoding into nlp
        doc = nlp(encode)
        doc2 = nlp2(particular_word)
        ents = [(e.text, e.ent_iob_, e.ent_type_, e.pos_) for e in doc]
        ents2 = [(f.text, f.ent_iob_, f.ent_type_, f.pos_) for f in doc2]

        label = []
        pos = []
        if ents2[0][2] != '':
            label.append(ents2[0][2])
        
        for numents in range(len(ents)):
            if ents[numents][2] != '':
                label.append(ents[numents][2])
            pos.append(ents[numents][3])
        

        # Filtering
        if label != []:
            label_counter = Counter(label)
            label_most_common = (label_counter.most_common(1))[0][0]
            
            # checking the label itself and then labeling it
            if label_most_common == 'PERSON':
                row['NER Label'] = 'B-PER'
            elif label_most_common == 'FAC':
                row['NER Label'] = 'B-LOC'
            elif label_most_common == 'NORP':
                row['NER Label'] = 'B-ORG'
            elif label_most_common == 'ORG':
                row['NER Label'] = 'B-ORG'
            elif label_most_common != 'MONEY' or label_most_common != 'QUANTITY':
                row['NER Label'] = 'B-MISC'
    
        else: 
            # Looking through pos taggings
            if 'PROPN' in pos:
                row['NER Label'] = 'B-MISC'
            elif 'NOUN' in pos:
                row['NER Label'] = 'B-MISC'
            

        
        

In [94]:
change_everything.to_csv('logs/changed_everything.tsv', sep='\t', index=False)

In [95]:
change_everything

,Word,POS Label,NER Label,Tweet
0,poutine,PROPN,B-MISC,rt @user1787 : i asked siri to remind me about...
1,friday,PROPN,B-MISC,rt @user912 : you know it 's friday when the p...
2,@user444,PROPN,B-MISC,mikel obi & his girl olga diyachenko spend hol...
3,@user1598,PROPN,B-MISC,""" @user1598 : super junior ’s siwon transforms..."
4,gospel,PROPN,B-MISC,""" do n't move on from the gospel , move into t..."
...,...,...,...,...
1013,@user70,PROPN,B-PER,rt @user2041 : @user70 happy birthday man ! le...
1014,@user660,PROPN,B-PER,rt @user1157 : happy birthday @user660 ! url1209
1015,@user2046,PROPN,B-PER,@user2046 further you do the swedish social se...
1016,@user2242,PROPN,B-PER,@user2046 further you do the swedish social se...
